In [ ]:
import os
import concurrent.futures
import requests
from pydub import AudioSegment
import pickle

In [ ]:
# Constants for API endpoint and API key
API_ENDPOINT = "your_asr_api_endpoint"
API_KEY = "your_api_key"

In [ ]:
# Function to transcribe a single audio file
def transcribe_audio(file_path):
    # Check cache for previously transcribed results
    cache_file = file_path + ".cache"
    if os.path.exists(cache_file):
        with open(cache_file, 'rb') as f:
            return pickle.load(f)

    # Convert audio file to WAV format
    audio = AudioSegment.from_file(file_path)
    wav_file = os.path.splitext(file_path)[0] + ".wav"
    audio.export(wav_file, format="wav")

    # Send request to ASR API
    headers = {"Authorization": "Bearer " + API_KEY}
    files = {"file": open(wav_file, "rb")}
    response = requests.post(API_ENDPOINT, headers=headers, files=files)

    # Retrieve transcription from response
    if response.status_code == 200:
        transcription = response.json()["transcription"]
        # Cache the result
        with open(cache_file, 'wb') as f:
            pickle.dump(transcription, f)
        return transcription
    else:
        return "Error: Unable to transcribe audio"

In [ ]:
# Function to transcribe a batch of audio files asynchronously
def process_batch(audio_files):
    transcribed_texts = {}
    with concurrent.futures.ThreadPoolExecutor() as executor:
        future_to_file = {executor.submit(transcribe_audio, file): file for file in audio_files}
        for future in concurrent.futures.as_completed(future_to_file):
            file = future_to_file[future]
            try:
                transcribed_texts[os.path.basename(file)] = future.result()
            except Exception as e:
                transcribed_texts[os.path.basename(file)] = str(e)
    return transcribed_texts

In [ ]:
# Example
if __name__ == "__main__":
    audio_files = ["audio1.mp3", "audio2.wav", "audio3.wav"]
    transcriptions = process_batch(audio_files)
    print(transcriptions)